# Clustering comparison

This notebook compares two clustering methods. The first clustering is named fastclust, and creates prototypes from the set of starting structures. The second clustering is a hierarchical ascendant clustering with linkage function as the radius of the smallest enclosing ball, the prototypes are the centers of the balls.

The first comparison is done on the number of clusters obtained. For this one it is not necessary to run the second clustering on your computer, we give the numbers here.

In [1]:
first_clust = {'AAA': 2589, 'AAC': 2063, 'ACA': 1810, 'ACC': 1781, 'CAA': 2154, 'CAC': 1545, 'CCA': 1934, 'CCC': 2515}
sec_clust = {'AAA': 1012, 'AAC': 712, 'ACA': 647, 'ACC': 661, 'CAA': 782, 'CAC': 566, 'CCA': 703, 'CCC': 839}

In [4]:
print("Ratio from first on second:")
for motif in first_clust.keys():
    print(motif + " = " + str(first_clust[motif]/sec_clust[motif]))

Ratio from first on second:
AAA = 2.558300395256917
AAC = 2.8974719101123596
ACA = 2.7975270479134466
ACC = 2.6944024205748867
CAA = 2.7544757033248084
CAC = 2.7296819787985864
CCA = 2.7510668563300142
CCC = 2.9976162097735397
